In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import fileio
import keys
import evaluation
import full_model
import data_aug
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os

In [2]:
DATA_DIR = './data/working'

labels = pd.read_pickle("{}/labels.pkl".format(DATA_DIR))

with np.load("{}/splits.npz".format(DATA_DIR)) as splits:
    train_idx = splits['train_idx']
    test_idx = splits['test_idx']

X = np.load("{}/X_cqt.npz".format(DATA_DIR))['X']
X_train = X[train_idx, :]
X_test = X[test_idx, :]

Y = np.load("{}/Y.npz".format(DATA_DIR))['Y']
Y_train = Y[train_idx, :]
Y_test = Y[test_idx, :]

aug = np.load("{}/data_aug.npz".format(DATA_DIR))

with np.load("{}/data_aug.npz".format(DATA_DIR)) as aug:
    X_train = np.vstack((X_train, aug['X']))
    Y_train = np.vstack((Y_train, aug['Y']))

X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()

Y_train = torch.from_numpy(np.argmax(Y_train, axis=1))
Y_test = torch.from_numpy(np.argmax(Y_test, axis=1))

KeyboardInterrupt: 

In [ ]:
import full_model
import model_tester
import importlib
importlib.reload(full_model)
importlib.reload(model_tester)

os.environ["CUDA_VISIBLE_DEVICES"]="2"
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

losses = model_tester.train_model(X_train, Y_train, X_test, Y_test, full_model.ConvBiLstm, weight_decay=0.0005,
                         device=device)

ConvBiLstm(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(10, 10, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(10, 10, kernel_size=(10, 10), stride=(2, 2), padding=(4, 4))
  (dense): Conv2d(10, 48, kernel_size=(72, 1), stride=(1, 1))
  (lstm): GRU(48, 48, num_layers=2, batch_first=True, bidirectional=True)
  (conv_pool): Conv2d(1, 10, kernel_size=(1, 75), stride=(1, 1))
  (fc1): Linear(in_features=96, out_features=48, bias=True)
  (fc3): Linear(in_features=48, out_features=24, bias=True)
)
epoch 0 loss: 3.1699487498630163
Train accuracy = 0.12732344970508527
Test accuracy = 0.1496413199426112


In [7]:
import IPython.display as ipd
ipd.Audio('/disk/scratch/lmd_matched_mp3/A/A/A/TRAAAZF12903CCCF6B.mp3')

In [55]:
import importlib
importlib.reload(fileio)
fileio.load_all_data(datasets=['msd'], chunk_start_nr=32, max_records=100)

Chunk directory exists data/working/chunks.
Shall I clean it (y/n)? >n
Chunks will be saved in: data/working/chunks
Processing MSD files ====================
Processing chunk 32 (size 100)
Writing labels
Labels saved in: data/working
FINISHED IMPORT!


In [18]:
DATA_DIR = './data/working'

y_labels = pd.read_pickle("{}/labels_million.pkl".format(DATA_DIR))

chunk = np.load('data/working/chunks/chunk0032.npz')
X = chunk['X']

import importlib
import utils
importlib.reload(utils)
from IPython import display

In [21]:
idx = np.random.choice(X.shape[0])
display.display(y_labels.loc[idx])
utils.play_from_label(y_labels.loc[idx])

chunk_idx                                                    23
chunk_nr                                                     32
filepath      data/raw/lmd_matched_mp3/A/B/L/TRABLTP128F930D...
genre                                                         A
key                                                           3
key_shift                                                     0
key_str                                                 C major
raw                                                           1
time_shift                                                    1
majmin                                                    major
dataset                                                 million
Name: 23, dtype: object